In [1]:
from google.colab import drive
drive.mount('/content/drive')

%cd ./drive/MyDrive/esc/prepare

Mounted at /content/drive
/content/drive/MyDrive/esc/prepare


In [2]:
%load_ext autoreload
%autoreload 2

import os, sys
import torch
import torch.nn as nn
import torch.nn.functional as F
sys.path.append('..')

from tqdm import tqdm
from network import ResNet
from dataset import InputPipeLineBuilder

In [3]:
num_epochs = 100
batch_size = 256

device = 'cuda' if torch.cuda.is_available() else 'cpu'
lr = 1e-3
weight_decay = 0.05

model = ResNet(head_input_dim=512).to(device)

for layer in model.modules():
  layer.requires_grad_ = True

loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=weight_decay)

sch1 = torch.optim.lr_scheduler.ConstantLR(optimizer, factor=1.0, total_iters=10)
sch2 = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.1)

scheduler = torch.optim.lr_scheduler.SequentialLR(
    optimizer,
    schedulers=[sch1, sch2],
    milestones=[10]
)

Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 208MB/s]


In [4]:
input_pipeline_builder = InputPipeLineBuilder(batch_size=batch_size, select_forget_concept=True, dataset='cifar100')

train_dataloader = input_pipeline_builder.get_dataloader_for_unlearn(is_retain=True, subset='train')
valid_dataloader = input_pipeline_builder.get_dataloader_for_unlearn(is_retain=True, subset='valid')
test_dataloader = input_pipeline_builder.get_dataloader_for_unlearn(is_retain=True, subset='test')

In [5]:
for epoch in range(num_epochs):
    losses = []
    model.train()
    for batch in tqdm(train_dataloader):
        train_x, train_y = batch
        logits = model(train_x.to(device))

        loss = loss_fn(logits, train_y.to(device))
        loss.backward()

        optimizer.step()
        optimizer.zero_grad()

        losses.append(loss.cpu().item())

    print(f"\tavg loss at epoch: {epoch+1}/{num_epochs}: {sum(losses) / len(losses):.4f}")

    model.eval()
    losses = []
    rcorrect, total = 0, 0
    for batch in valid_dataloader:
      valid_x, valid_y = batch
      logits = model(valid_x.to(device))

      loss = loss_fn(logits, valid_y.to(device))
      losses.append(loss.cpu().item())
      pred_labels = torch.argmax(logits, dim=1)
      num_preds = torch.sum((pred_labels == valid_y.to(device))).item()

      rcorrect += num_preds
      total += valid_x.shape[0]
    print(f"\tvalid acc at epoch: {epoch+1}/{num_epochs} : {rcorrect / total:.4f}")
    print(f"\tavg loss at epoch: {epoch+1}/{num_epochs}: {sum(losses) / len(losses):.4f}")

    scheduler.step()

100%|██████████| 140/140 [00:11<00:00, 11.70it/s]


	avg loss at epoch: 1/100: 2.6964
	valid acc at epoch: 1/100 : 0.3812
	avg loss at epoch: 1/100: 2.4053


100%|██████████| 140/140 [00:10<00:00, 13.24it/s]


	avg loss at epoch: 2/100: 1.8565
	valid acc at epoch: 2/100 : 0.4389
	avg loss at epoch: 2/100: 2.1426


100%|██████████| 140/140 [00:10<00:00, 13.37it/s]


...

	avg loss at epoch: 98/100: 0.0009
	valid acc at epoch: 98/100 : 0.6050
	avg loss at epoch: 98/100: 2.4497


100%|██████████| 140/140 [00:10<00:00, 13.35it/s]


	avg loss at epoch: 99/100: 0.0009
	valid acc at epoch: 99/100 : 0.6067
	avg loss at epoch: 99/100: 2.4636


100%|██████████| 140/140 [00:09<00:00, 14.11it/s]


	avg loss at epoch: 100/100: 0.0010
	valid acc at epoch: 100/100 : 0.6078
	avg loss at epoch: 100/100: 2.4553


In [ ]:
torch.save(model.state_dict(), './resnet_18_retrained.pth')